# 1. 크롤링과 함께 쓰는 mongoDB 예제

### cine21 인물 랭킹 알아내기
  - [씨네21-인물 순위](http://www.cine21.com/rank/person/)
  - 조회를 눌렀을 때, Go to Network -> content 의 Request URL/Method 와 Form 데이터 알아내기
    - Request URL: http://www.cine21.com/rank/person/content
    - Request Method: POST
    - Form Data
      - section:actor
      - period_start:2021-10
      - gender:all
      - page:1
    - 하단부 페이지를 누를 때마다, Form Data 의 page 값이 바뀜

In [1]:
import requests as req
import re
import datetime
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
import pymongo

In [2]:
# 몽고DB 서버 연결
username = 'ubuntu'
password = '1234'
conn = pymongo.MongoClient('mongodb://%s:%s@127.0.0.1:27017' % (username, password))

In [3]:
# db, collection 연결(없으면 생성)
# python과 연동되었을 때, db와 collection의 경우, document insert 싯점에 생성됨
mongodb = conn.cine21

# collection 객체 변수 - 몽고 db 컬렉션 이름(없으면 안 만들어짐)
actor_collection = mongodb.actor_collection

In [4]:
# actor 정보가 있는지 확인하기
actor_list = actor_collection.find()
for actor in actor_list:
    print(actor['actor'])

In [5]:
# request url
cine21_url = 'http://www.cine21.com/rank/person/content'
month = "2021-10"
year = "2020-11"

# post 방식으로 데이터를 가져오기 위한 request 시 필요 정보
# 브라우저 개발자 모드의 network tab의 form 정보
conditions = dict()
conditions['section'] = 'actor'
conditions['period_start'] = month
conditions['gender'] = 'all'
conditions['page'] = 1

# post 방식으로 request 요청
response = req.post(cine21_url, data = conditions)

In [ ]:
response.content

In [7]:
#html 문서안 soup 객체 생성
soup = bs(response.content, 'html.parser')
soup

 <ul class="people_list">
<li class="people_li">
<a href="/db/person/info/?person_id=74826"><img alt="" class="people_thumb" src="https://image.cine21.com/resize/cine21/person/2019/0828/11_54_52__5d65ecfc13ca6[X145,145].jpg" target="_blank"/></a>
<div class="name"><a href="/db/person/info/?person_id=74826">박정민(1편)</a></div>
<ul class="num_info">
<li><span class="tit">흥행지수</span><strong>160,806</strong></li>
<!--
						<li><a href="#" class="btn_graph"><span class="ico"></span><span>흥행성적<br />그래프로 보기</span></a></li>
						-->
</ul>
<!-- 영화포스터는 최대 5개까지만 -->
<ul class="mov_list">
<li>
<a href="/movie/info/?movie_id=57227">
<img alt="" class="thumb" src="https://image.cine21.com/resize/cine21/poster/2021/0928/10_31_26__6152706e344e6[X85,120].jpg" target="_blank"/>
<span>기적</span>
</a>
</li>
</ul>
<!-- 순위 --><span class="grade">1</span>
</li>
<li class="people_li">
<a href="/db/person/info/?person_id=25123"><img alt="" class="people_thumb" src="https://image.cine21.com/resize/cine21/person/

In [36]:
actors = soup.select("li.people_li div.name")
actors

[<div class="name"><a href="/db/person/info/?person_id=74826">박정민(1편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=25123">이성민(1편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=66313">임윤아(1편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=70020">노회찬(1편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=96165">이수경(1편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=32987">박영남(1편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=74727">김환진(1편)</a></div>]

In [37]:
for actor in actors:
    print(actor.select("a"))

[<a href="/db/person/info/?person_id=74826">박정민(1편)</a>]
[<a href="/db/person/info/?person_id=25123">이성민(1편)</a>]
[<a href="/db/person/info/?person_id=66313">임윤아(1편)</a>]
[<a href="/db/person/info/?person_id=70020">노회찬(1편)</a>]
[<a href="/db/person/info/?person_id=96165">이수경(1편)</a>]
[<a href="/db/person/info/?person_id=32987">박영남(1편)</a>]
[<a href="/db/person/info/?person_id=74727">김환진(1편)</a>]


In [10]:
# 정규 표현식 참고 : https://regexr.com/
# 문자, 숫자를 찾음 : \w
# 특수 기호 () 문자로 인식 : \( \)

# 2. 반복 표현 ?, *, +
# + : 앞 문자가 1번 또는 그 이상 반복되는 패턴
# ? : 앞 문자가 0번 또는 1번 표시되는 패턴(없어도 되고, 한번 있어도 되는 패턴)
# * : 앞 문자가 0번 또는 그이상 반복되는 패턴
import re
test_data = '마동석(17편)'
#re.sub('정규표현식', '변경데이터', test_data)
# "(숫자)" 부분을 찾아서 "" 공백으로 만들어 버린다.
print(re.sub("\(\w+\)", " 미국인", test_data))

마동석 미국인


In [11]:
# [실습]
# 한 페이지 내의 배우이름만 가져오기
import re

def remove_num(name):
    return re.sub("\(\w+\)", "", name)

actor_lst = list()
for actor in actors:
    # 연산은 한번만하고 그 값을 출력, 리스트에 추가한다.
    temp = remove_num(actor.getText())
    print( temp )
    actor_lst.append(temp)

print('-'*62)
actor_lst

박정민
이성민
임윤아
노회찬
이수경
박영남
김환진
--------------------------------------------------------------


['박정민', '이성민', '임윤아', '노회찬', '이수경', '박영남', '김환진']

### 각 배우별 상세 정보를 document에 넣고 싶다.
* 각 배우별 상세 정보를 별도 컬럼으로 만들려했더니, 각 배우별 상세 정보 항목이 다르다!
* 모든 상세 정보 항목을 컬럼으로 만들고, 각 컬럼에 매칭되는 컬럼값을 넣기가 쉽지 않다. 코드도 복잡하고!
* Mongodb는 NoSQL -> 통째로 집어넣자.!

* embedded document
  - document 의 컬럼값으로 document를 넣을 수 있다.

In [12]:
actors

[<div class="name"><a href="/db/person/info/?person_id=74826">박정민(1편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=25123">이성민(1편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=66313">임윤아(1편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=70020">노회찬(1편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=96165">이수경(1편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=32987">박영남(1편)</a></div>,
 <div class="name"><a href="/db/person/info/?person_id=74727">김환진(1편)</a></div>]

In [13]:
def remove_span(tag):
    return re.sub('<span.*?>.*?<\/span>',"", tag)

def remove_li(tag):
    return re.sub('<.*?>',"", tag)

In [14]:
detail = []
for actor in actors:
    href = actor.select_one('a').attrs["href"]
    url = "http://www.cine21.com" + href

    res = req.get(url)

    # 배우 상세 정보 저장하는 soup 객체(html 객체)
    soup_actor = bs(res.content, "html.parser")
    info = soup_actor.select_one("ul.default_info")
    # print(info)

    # 배우 상세 정보를 리스트로 반한환 리턴받음.
    db = dict()
    actor_details = info.select('li')
    for actor_detail in actor_details:

        # 방법 1.
        key = actor_detail.select_one('span.tit').get_text(strip=True)
        value = remove_li(remove_span(str(actor_detail))).replace('\n',", ").strip(', ')
        db[key] = value

        # 방법 2. 상세 정보 key 추출
        # c = actor_detail
        # print(c)
        # a = actor_detail.select_one('span').extract() # get_text()
        # print(a)
        # b = actor_detail.get_text()
        # print(b)

        # print("-"*20)
    detail.append(db)

detail

[{'직업': '배우',
  '생년월일': '1987-02-25',
  '성별': '남',
  '신장/체중': '178cm, 63kg',
  '학교': '한국예술종합학교 영상원 연극원 연기과'},
 {'직업': '배우', '생년월일': '1968-10-15', '성별': '남', '신장/체중': '178cm'},
 {'다른 이름': '소녀시대; girlsgeneration; girls generation',
  '직업': '가수',
  '생년월일': '1990-05-30',
  '성별': '여',
  '홈페이지': 'https://www.instagram.com/yoona__lim/',
  '소속사': 'SM엔터테인먼트'},
 {'생년월일': '1956-08-31',
  '사망': '2018-07-23',
  '성별': '남',
  '홈페이지': 'https://twitter.com/hcroh, https://www.facebook.com/omyChans'},
 {'직업': '배우', '생년월일': '1996-10-24', '성별': '여'},
 {'직업': '성우', '생년월일': '1946-10-08', '성별': '여'},
 {'직업': '성우', '생년월일': '1952-10-03', '성별': '남'}]

### 흥행지수 뽑기

In [15]:
hits = soup.select('strong')
actors_hit_info = list()
for idx in range(len(hits)):
    print('배우이름 :',actor_lst[idx])
    print("흥행지수 :",hits[idx].get_text())
    print("-"*17)
    actors_hit_info.append(hits[idx].get_text())
actors_hit_info


배우이름 : 박정민
흥행지수 : 160,806
-----------------
배우이름 : 이성민
흥행지수 : 120,606
-----------------
배우이름 : 임윤아
흥행지수 : 80,402
-----------------
배우이름 : 노회찬
흥행지수 : 51,374
-----------------
배우이름 : 이수경
흥행지수 : 40,202
-----------------
배우이름 : 박영남
흥행지수 : 27,566
-----------------
배우이름 : 김환진
흥행지수 : 24,120
-----------------


['160,806', '120,606', '80,402', '51,374', '40,202', '27,566', '24,120']

### 각 배우별 출연 영화를 document에 저장하고 싶다.
  - 출연 영화는 한 개가 될 수도 있고, 여러 개가 될 수도 있음
  - 파이썬은 리스트, mongodb document는 컬럼에 배열(array)로 넣으면 됨 

In [16]:
movie_lst = list()
movies = soup.select('li.people_li ul.mov_list a span')
for idx in range(len(movies)):
    actor_movie = dict()
    title = movies[idx].get_text()
    movie_lst.append(title)
movie_lst    

['기적',
 '기적',
 '기적',
 '노회찬6411',
 '기적',
 '극장판 짱구는 못말려: 격돌! 낙서왕국과 얼추 네 명의 용사들',
 '극장판 짱구는 못말려: 격돌! 낙서왕국과 얼추 네 명의 용사들']

### insert_one() 로 하나씩 데이터 입력하기 (반복문과 함께 사용하면, 여러 데이터를 넣을 수 있음)

- actor_list: 배우 이름
- actor_details: 배우 상세 정보
- actor_rate: 흥행 지수
- date: 기준월
- movie_list: 출연 영화 리스트!

In [17]:
actor_lst

['박정민', '이성민', '임윤아', '노회찬', '이수경', '박영남', '김환진']

In [18]:
for num, actor in enumerate(actor_lst):
    actor_collection.insert_one(
        {
            "actor":actor,
            "actor_details":detail[num],
            "actor_hit":actors_hit_info[num],
            "date":month,
            "movie_list":movie_lst[num]
        }
    )

In [19]:
actors_info = list()
for num, actor in enumerate(actor_lst):
    actors_info.append(
        {
            "actor":actor,
            "actor_details":detail[num],
            "actor_hit":actors_hit_info[num],
            "date":month,
            "movie_list":movie_lst[num]
        }
    )
print(actors_info)
actor_collection.insert_many(actors_info)

[{'actor': '박정민', 'actor_details': {'직업': '배우', '생년월일': '1987-02-25', '성별': '남', '신장/체중': '178cm, 63kg', '학교': '한국예술종합학교 영상원 연극원 연기과'}, 'actor_hit': '160,806', 'date': '2021-10', 'movie_list': '기적'}, {'actor': '이성민', 'actor_details': {'직업': '배우', '생년월일': '1968-10-15', '성별': '남', '신장/체중': '178cm'}, 'actor_hit': '120,606', 'date': '2021-10', 'movie_list': '기적'}, {'actor': '임윤아', 'actor_details': {'다른 이름': '소녀시대; girlsgeneration; girls generation', '직업': '가수', '생년월일': '1990-05-30', '성별': '여', '홈페이지': 'https://www.instagram.com/yoona__lim/', '소속사': 'SM엔터테인먼트'}, 'actor_hit': '80,402', 'date': '2021-10', 'movie_list': '기적'}, {'actor': '노회찬', 'actor_details': {'생년월일': '1956-08-31', '사망': '2018-07-23', '성별': '남', '홈페이지': 'https://twitter.com/hcroh, https://www.facebook.com/omyChans'}, 'actor_hit': '51,374', 'date': '2021-10', 'movie_list': '노회찬6411'}, {'actor': '이수경', 'actor_details': {'직업': '배우', '생년월일': '1996-10-24', '성별': '여'}, 'actor_hit': '40,202', 'date': '2021-10', 'movie_list': '기적'}, 

## Document Key명 변경 방법
* ## actor -> actor_name으로 변경

In [20]:
# actor_collection.update_many({}, {"$rename":{"원래키":"변경할 키"}})
actor_collection.update_many(
    {},
    {"$rename":{"actor":"actor_name"}}
)

* ## actor_details -> actor_info 로 변경해본다.

In [21]:
actor_collection.update_many(
    {},
    {"$rename":{"actor_details":"actor_info"}}
)

In [ ]:
# collection drop
# actor_collection.drop()

In [ ]:
# collection 찾기
docs = actor_collection.find()
docs

In [ ]:
for doc in docs:
    print(doc)

### Dictionary 타입으로 만들어서 한번에 insert_many() 로 데이터 입력하기

### Update (컬럼명 변경 예제)

### 컬렉션 객체 이름도 바꿀 수 있겠지요
actor_collection -> actors_info 로 변경